In [1]:
# !pip install selenium
# !pip install webdriver-manager  # no need to download any driver file

In [3]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager

''' previously it was used 
# PATH = "C:Program Files (x86)/msedgedriver.exe"
# driver = webdriver.Edge()
'''

driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
driver.maximize_window()    # to maximize the web page
driver.get('https://www.screener.in/')


In [7]:
# used if it close automatically
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.edge.options import Options

edge_options = Options()
edge_options.add_experimental_option("detach",True)

driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()),options=edge_options) # it was closing on its own to not able to do it 
driver.get('https://www.screener.in/')


### Different methods

In [40]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys # to use enter 
import time


driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
# driver.maximize_window()    # to maximize the web page
driver.get('https://www.screener.in/')

# xpath Method
input = driver.find_element(By.XPATH,'/html/body/main/div[2]/div/div/div/input')
input.send_keys("Vedanta")
input.send_keys(Keys.RETURN)

# classmethod
# input = driver.find_element(By.CLASS_NAME,'u-full-width')
# input.send_keys("Vedanta")

# tagname method
# input = driver.find_element(By.TAG_NAME,'input')
# input.send_keys("Vedanta")

# name method
# driver.get("https://www.google.com")
# input = driver.find_element(By.NAME,'q').send_keys("Ferrari")

# Partial link text
# driver.find_element(By.PARTIAL_LINK_TEXT,"Gmail").click()

# Link text
# driver.find_element(By.LINK_TEXT,'Login').click()


# For multiple rows

In [125]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys # to use enter 
import time

username = "melibi8600@exclussi.com"
password = "screener@12345"

driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
# driver.maximize_window()    # to maximize the web page
driver.get('https://www.screener.in/')

driver.find_element(By.XPATH,"/html/body/nav/div[2]/div/div/div/div[2]/div[2]/a[1]").click()
driver.find_element(By.NAME,"username").send_keys(username)
time.sleep(1)
driver.find_element(By.NAME,"password").send_keys(password)
time.sleep(1)
driver.find_element(By.XPATH,"/html/body/main/div[2]/div[2]/form/button").click()
time.sleep(1)
driver.find_element(By.XPATH,"/html/body/div/div[2]/main/div[1]/div[2]/a[1]").click()

data = {}

tr_rows = driver.find_elements(By.XPATH,"/html/body/main/div[3]/div[3]/table/tbody/tr")
req_rows = len(tr_rows)

for cols in range(2,19):
    head = driver.find_element(By.XPATH,f'/html/body/main/div[3]/div[3]/table/tbody/tr[1]/th[{cols}]').text
    data[head]=[]
    head=''

for rows in range(2,req_rows+1):
    for cols in range(2,19):
        head = driver.find_element(By.XPATH,f'/html/body/main/div[3]/div[3]/table/tbody/tr[1]/th[{cols}]').text
        if cols>=2:
            value=driver.find_element(By.XPATH,f'/html/body/main/div[3]/div[3]/table/tbody/tr[{rows}]/td[{cols+1}]').text
        data[head].append(value)
        head,value='',''

time.sleep(1)
print("Your data is Scrap from Screener.in")
driver.close()

In [126]:
import psycopg2

conn = None
cur = None

try:
    conn = psycopg2.connect(host='localhost', dbname='postgres',user='postgres',password="mypass",port=5431)
    cur = conn.cursor()
    cols = [col.replace(" ",'_') for col,_ in data.items()]
    cols = [col.replace('%','PERCENT') for col in cols]
    cols = [col.replace('.','') for col in cols]
    cols = [col.replace('/','BY') for col in cols]
    rows = [v for _,v in data.items()]
    cur.execute("select name from stocks")
    conn.commit()
    company_name = []
    for i in cur.fetchall():
        company_name.append(i[0])

    row_data = []
    li =[]

    for row in range(len(rows)):
        for col in range(len(rows[row])):
            for r in range(len(rows)):
                if rows[r][col] == '':
                    li.append(0)
                    continue
                li.append(rows[r][col])
            row_data.append(li)
            li=[]
        break
    
    for row in row_data:
        if row[0] not in company_name:
            print(row[0])
            cur.execute(f"""
                insert into stocks ({', '.join(cols)})
                values ({', '.join(['%s'] * len(cols))})
            """,row)
            conn.commit()
            print("Inserted into stocks table.")
        

except Exception as e:
    print(e)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

Titan Company
Inserted into stocks table.


In [106]:
data 
cols = [col.replace(" ",'_') for col,_ in data.items()]
cols = [col.replace('%','PERCENT') for col in cols]
cols = [col.replace('.','') for col in cols]
cols = [col.replace('/','BY') for col in cols]
rows = [v for k,v in data.items()]
# print(len(cols),len(rows))
# print(cols)
    # print(rows)
company_name = rows[0]
# if 
row_data = []
li =[]
for row in range(len(rows)):
    for col in range(len(rows[row])):
        for r in range(len(rows)):
            if rows[r][col] == '':
                li.append('NULL')
                continue
            li.append(rows[r][col])
        row_data.append(li)
        li=[]
    break
print(row_data)

[['ITC Hotels', '199.25', '41467.40', '0.00', '0.74', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', 'NULL', '2596732.00', 'NULL', 'NULL', 'NULL', '54.18'], ['ITC', '420.05', '525655.64', '3.26', '5013.18', '-8.40', '9.28', '37.47', '75160.56', '28.43', '7.73', '-2.09', '3710169.00', '12.89', '16.12', '18.01', '58.08'], ['Jubilant Food.', '688.00', '45397.31', '0.18', '43.24', '-21.97', '56.06', '11.20', '7611.34', '12.98', '42.24', '4.08', '363143.00', '19.52', '5.61', '31.32', '57.96'], ['Vedanta', '384.00', '150158.96', '11.34', '4876.00', '76.20', '10.06', '20.91', '148022.00', '10.49', '1.28', '138.51', '1999975.00', '17.76', '33.58', '5.08', '26.48'], ['Infosys', '1423.05', '591030.16', '2.66', '6822.00', '11.46', '7.58', '39.99', '159988.00', '31.83', '4.44', '13.36', '2542766.00', '15.22', '66.60', '13.32', '26.53']]


In [ ]:
# cols = [col.replace(" ",'_') for col,_ in data.items()]
# cols = [col.replace('%','PERCENT') for col in cols]
# cols = [col.replace('.','') for col in cols]
# cols = [col.replace('/','BY') for col in cols]
# for i in cols:
#     print(f'{i} numeric,')

Name numeric,
CMP_Rs numeric,
Mar_Cap_RsCr numeric,
Div_Yld_PERCENT numeric,
NP_Qtr_RsCr numeric,
Qtr_Profit_Var_PERCENT numeric,
Qtr_Sales_Var_PERCENT numeric,
ROCE_PERCENT numeric,
Sales_RsCr numeric,
ROE_PERCENT numeric,
Sales_growth_PERCENT numeric,
Profit_growth_PERCENT numeric,
No_of_Share_Holders numeric,
Sales_Var_3Yrs_PERCENT numeric,
EPS_12M_Rs numeric,
EV_BY_EBITDA numeric,
RSI numeric,


In [25]:
for i,j in data.items():
    print(i,j)

Name ['Infosys', 'Vedanta']
CMP Rs. [1403.7, 370.55]
Mar Cap Rs.Cr. [582993.59, 144899.43]
Div Yld % [2.71, 11.74]
NP Qtr Rs.Cr. [6822.0, 4876.0]
Qtr Profit Var % [11.46, 76.2]
Qtr Sales Var % [7.58, 10.06]
ROCE % [39.99, 20.91]
Sales Rs.Cr. [159988.0, 148022.0]
ROE % [31.83, 10.49]
Sales growth % [4.44, 1.28]
Profit growth % [13.36, 138.51]
No. of Share Holders [2542766.0, 1999975.0]
Sales Var 3Yrs % [15.22, 17.76]
EPS 12M Rs. [66.6, 33.58]
EV / EBITDA [13.13, 4.95]
RSI [26.53, 26.48]


In [23]:
dat = {'Name': []}
for i in range(3):
    dat['Name'].append(i)
print(dat)

{'Name': [0, 1, 2]}


# One single row

In [55]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys # to use enter 
import time
import pandas as pd
import numpy as np

username = "melibi8600@exclussi.com"
password = "screener@12345"

driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
# driver.maximize_window()    # to maximize the web page
driver.get('https://www.screener.in/')

driver.find_element(By.XPATH,"/html/body/nav/div[2]/div/div/div/div[2]/div[2]/a[1]").click()
driver.find_element(By.NAME,"username").send_keys(username)
time.sleep(2)
driver.find_element(By.NAME,"password").send_keys(password)
time.sleep(1)
driver.find_element(By.XPATH,"/html/body/main/div[2]/div[2]/form/button").click()
time.sleep(3)
driver.find_element(By.XPATH,"/html/body/div/div[2]/main/div[1]/div[2]/a[1]").click()

li_d=[]
data = {}

for i in range(2,19):
    head = driver.find_element(By.XPATH,f'/html/body/main/div[3]/div[3]/table/tbody/tr[1]/th[{i}]').text 
    if i >= 2: 
        value=driver.find_element(By.XPATH,f'/html/body/main/div[3]/div[3]/table/tbody/tr[2]/td[{i+1}]').text
        if value.isalpha():
            pass
        else:
            value=float(value)
    data[head]=value
    head,value='',''
    
# data.append(data)
df = pd.DataFrame(data,index=[1])
print(df)
# print(data)
# df.to_csv('stock.csv',index=False)
time.sleep(5)
driver.close()

      Name  CMP Rs.  Mar Cap Rs.Cr.  Div Yld %  NP Qtr Rs.Cr.  \
1  Infosys   1403.7       582993.59       2.71         6822.0   

   Qtr Profit Var %  Qtr Sales Var %  ROCE %  Sales Rs.Cr.  ROE %  \
1             11.46             7.58   39.99      159988.0  31.83   

   Sales growth %  Profit growth %  No. of Share Holders  Sales Var 3Yrs %  \
1            4.44            13.36             2542766.0             15.22   

   EPS 12M Rs.  EV / EBITDA    RSI  
1         66.6        13.13  26.53  


In [ ]:
import psycopg2

conn = None
cur = None

try:
    conn = psycopg2.connect(host='localhost', dbname='postgres',user='postgres',password="mypass",port=5431)
    cur = conn.cursor()
    cols = [col.replace(" ",'_') for col,_ in data.items()]
    cols = [col.replace('%','PERCENT') for col in cols]
    cols = [col.replace('.','') for col in cols]
    cols = [col.replace('/','BY') for col in cols]
    rows = [v for k,v in data.items()]
    print(len(cols),len(rows))
    print(cols)
    print(rows)
    # ind = []


    cur.execute(f"""
                insert into stocks ({', '.join(cols)})
                values ({', '.join(['%s'] * len(cols))})
            """,rows)
    print("done")
    conn.commit()
    print('Inserted into stocks')
except Exception as e:
    print(e)
finally:
    if cur is not None:
        cur.close()
    if conn is not None:
        conn.close()

17 17
['Name', 'CMP_Rs', 'Mar_Cap_RsCr', 'Div_Yld_PERCENT', 'NP_Qtr_RsCr', 'Qtr_Profit_Var_PERCENT', 'Qtr_Sales_Var_PERCENT', 'ROCE_PERCENT', 'Sales_RsCr', 'ROE_PERCENT', 'Sales_growth_PERCENT', 'Profit_growth_PERCENT', 'No_of_Share_Holders', 'Sales_Var_3Yrs_PERCENT', 'EPS_12M_Rs', 'EV_BY_EBITDA', 'RSI']
['Infosys', 1403.7, 582993.59, 2.71, 6822.0, 11.46, 7.58, 39.99, 159988.0, 31.83, 4.44, 13.36, 2542766.0, 15.22, 66.6, 13.13, 26.53]
done
Inserted into stocks


In [154]:
data

{'Name': 'Vedanta',
 'CMP Rs.': 370.5,
 'Mar Cap Rs.Cr.': 144879.88,
 'Div Yld %': 11.8,
 'NP Qtr Rs.Cr.': 4876.0,
 'Qtr Profit Var %': 76.2,
 'Qtr Sales Var %': 10.06,
 'ROCE %': 20.91,
 'Sales Rs.Cr.': 148022.0,
 'ROE %': 10.49,
 'Sales growth %': 1.28,
 'Profit growth %': 138.51,
 'No. of Share Holders': 1999975.0,
 'Sales Var 3Yrs %': 17.76,
 'EPS 12M Rs.': 33.58,
 'EV / EBITDA': 4.95,
 'RSI': 27.58}

In [149]:
cols = [col for col,_ in data.items()]
rows = [v for k,v in data.items()]
print(len(cols),len(rows))

17 17


In [155]:
z = ["Name","id"]
z

['Name', 'id']

In [131]:
# df['sp']=df['Sales Rs.Cr.']*df['Profit growth %']
# df.info()
df.drop(['sp'],axis=1)

,Name,CMP Rs.,Mar Cap Rs.Cr.,Div Yld %,NP Qtr Rs.Cr.,Qtr Profit Var %,Qtr Sales Var %,ROCE %,Sales Rs.Cr.,ROE %,Sales growth %,Profit growth %,No. of Share Holders,Sales Var 3Yrs %,EPS 12M Rs.,EV / EBITDA,RSI
1,Vedanta,370.5,144879.88,11.8,4876.0,76.2,10.06,20.91,148022.0,10.49,1.28,138.51,1999975.0,17.76,33.58,4.95,27.58


In [ ]:
d = {'Name': 'Vedanta', 'CMP Rs.': '370.75', 'Mar Cap Rs.Cr.': '144977.65', 'Div Yld %': '11.80', 'NP Qtr Rs.Cr.': '4876.00', 'Qtr Profit Var %': '76.20', 'Qtr Sales Var %': '10.06', 'ROCE %': '20.91', 'Sales Rs.Cr.': '148022.00', 'ROE %': '10.49', 'Sales growth %': '1.28', 'Profit growth %': '138.51', 'No. of Share Holders': '1999975.00', 'Sales Var 3Yrs %': '17.76', 'EPS 12M Rs.': '33.58', 'EV / EBITDA': '4.96', 'RSI': '27.58'}
for k,v in d.items():
    print(f'"{k}" float,')

In [178]:
for i in cols:
    print(f"{i} float,")

Name float,
CMP_Rs float,
Mar_Cap_RsCr float,
Div_Yld_PERCENT float,
NP_Qtr_RsCr float,
Qtr_Profit_Var_PERCENT float,
Qtr_Sales_Var_PERCENT float,
ROCE_PERCENT float,
Sales_RsCr float,
ROE_PERCENT float,
Sales_growth_PERCENT float,
Profit_growth_PERCENT float,
No_of_Share_Holders float,
Sales_Var_3Yrs_PERCENT float,
EPS_12M_Rs float,
EV_BY_EBITDA float,
RSI float,


In [55]:
'98'.isdigit()

True

In [1]:
rows, cols = 3,6
my_matrix = [([0]*cols) for i in range(rows)]
my_matrix

[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]

### Code to automate YT shorts scrolling action

```python
# Automate scrolling action for YT shorts

from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys # to use enter 
import time

driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))
# driver.maximize_window()    # to maximize the web page
driver.get('https://www.youtube.com/shorts/jxHyGE_WuVM')
time.sleep(5)

# input = driver.find_element(By.XPATH,'/html/body/main/div[2]/div/div/div/input')
input = driver.find_element(By.XPATH,'/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[3]/div[2]/ytd-reel-video-renderer[1]/div[1]/div[2]/div[1]/ytd-player/div/div/div[8]/button')
input.click()

for i in range(5):
    next=driver.find_element(By.XPATH,'/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[5]/div[2]/ytd-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]')
    next.click()
    time.sleep(5)


```